<a href="https://colab.research.google.com/github/shashithenuwara/IRWA_Project/blob/project/upload_from_pc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1.Data Collection**

In [24]:
#From local device
from google.colab import files

# Uploading the PDFs from local device
uploaded = files.upload()

# Saving the names of uploaded files
pdf_files = uploaded.keys()

Saving Breast-cancer-susceptibility&#x2014;A-new-look-at-.pdf to Breast-cancer-susceptibility&#x2014;A-new-look-at- (1).pdf
Saving Integrated-breast-cancer-genomics_ccell.pdf to Integrated-breast-cancer-genomics_ccell (1).pdf


**2. Data Preprocessing**

2.1 Text Extraction

In [3]:
#Installing required libraries for PDF text extraction, tokenization, normalization, and other NLP tasks
!pip install PyPDF2 pdfplumber spacy nltk
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 62.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
import pdfplumber

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text

# Loop through each uploaded PDF
extracted_texts = {}
for pdf_file in pdf_files:
    # Save the extracted text for each PDF file
    extracted_texts[pdf_file] = extract_text_from_pdf(pdf_file)

# Convert the dict_keys to a list first
first_file = list(extracted_texts.keys())[0]  # Get the first file name

# Access the extracted text of the first file
print(extracted_texts[first_file])

P R E V I E W S
Breast cancer susceptibility—A new look at an old model
A polygenic model in which many individually weak genes combine multiplicately to cause a 50-fold range of risk in the
population explains several puzzling aspects of familial breast cancer epidemiology,including the very high risk in some
families and the failure to identify important new genes since the discovery of BRCA1and BRCA2.
Germline mutations in BRCA1or BRCA2 model was developed by segregation several quite different genetic models
account for most large multiple-case analysis in a population-based series of provide a statistically satisfactory fit to
breast cancer families but only a few per- families and in a collection of multiple such family data. The original CASH
cent of unselected cases. It has been case families in which breast cancer model involved a rare highly penetrant
recognized since BRCA2 was cloned patients had been screened for BRCA1 dominant gene (Claus et al., 1991), and
that these and 

In [6]:
for pdf_file, text in extracted_texts.items():
    # Save the extracted text to a .txt file with the same name as the PDF
    with open(pdf_file.replace('.pdf', '.txt'), 'w') as txt_file:
        txt_file.write(text)

2.2 Data Cleaning

In [7]:
#To remove HTML tags
!pip install beautifulsoup4

In [8]:
import re
from bs4 import BeautifulSoup
from google.colab import files
import pdfplumber

# Step 3: Cleaning function to process text
def clean_text(text):
    # Removing HTML tags
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()

    # Removing special characters and punctuation (except basic punctuation)
    text = re.sub(r"[^a-zA-Z0-9\s,.!?'-]", '', text)

    # Removing extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Removing duplicate lines/sentences
    sentences = text.split('. ')
    unique_sentences = list(dict.fromkeys(sentences))
    cleaned_text = '. '.join(unique_sentences)

    return cleaned_text

In [12]:
# Step 4: Apply the cleaning function to the extracted text
cleaned_texts = {filename: clean_text(text) for filename, text in extracted_texts.items()}

# Step 5: Print cleaned text from one file (e.g., the first uploaded file)
first_file = list(cleaned_texts.keys())[0]  # Get the name of the first file
print(cleaned_texts[first_file])

P R E V I E W S Breast cancer susceptibilityA new look at an old model A polygenic model in which many individually weak genes combine multiplicately to cause a 50-fold range of risk in the population explains several puzzling aspects of familial breast cancer epidemiology,including the very high risk in some families and the failure to identify important new genes since the discovery of BRCA1and BRCA2. Germline mutations in BRCA1or BRCA2 model was developed by segregation several quite different genetic models account for most large multiple-case analysis in a population-based series of provide a statistically satisfactory fit to breast cancer families but only a few per- families and in a collection of multiple such family data. The original CASH cent of unselected cases. It has been case families in which breast cancer model involved a rare highly penetrant recognized since BRCA2 was cloned patients had been screened for BRCA1 dominant gene Claus et al., 1991, and that these and a f

2.3 Normalization

In [14]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

# Download required NLTK packages
nltk.download('punkt')
nltk.download('stopwords')

# Tokenization and normalization function
def preprocess_text(text):
    # Converting to lowercase and tokenizing
    tokens = word_tokenize(text.lower())

    # Removing punctuation marks and stop words
    tokens = [word for word in tokens if word.isalpha()]  # Keep only alphabetic tokens
    stop_words = set(stopwords.words('english'))  # Get English stop words
    tokens = [word for word in tokens if word not in stop_words]  # Remove stop words

    return tokens

# Assuming `pdf_texts` is a dictionary from the earlier text extraction step
# Preprocess each extracted text and store in a new dictionary
preprocessed_texts = {filename: preprocess_text(text) for filename, text in extracted_texts.items()}

# Print preprocessed tokens from the first file for verification
first_file = list(preprocessed_texts.keys())[0]  # Get the name of the first file
print(f"Preprocessed tokens for {first_file}:")
print(preprocessed_texts[first_file])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Preprocessed tokens for Breast-cancer-susceptibility&#x2014;A-new-look-at-.pdf:
['p', 'r', 'e', 'v', 'e', 'w', 'breast', 'cancer', 'new', 'look', 'old', 'model', 'polygenic', 'model', 'many', 'individually', 'weak', 'genes', 'combine', 'multiplicately', 'cause', 'range', 'risk', 'population', 'explains', 'several', 'puzzling', 'aspects', 'familial', 'breast', 'cancer', 'epidemiology', 'including', 'high', 'risk', 'families', 'failure', 'identify', 'important', 'new', 'genes', 'since', 'discovery', 'germline', 'mutations', 'model', 'developed', 'segregation', 'several', 'quite', 'different', 'genetic', 'models', 'account', 'large', 'analysis', 'series', 'provide', 'statistically', 'satisfactory', 'fit', 'breast', 'cancer', 'families', 'families', 'collection', 'multiple', 'family', 'data', 'original', 'cash', 'cent', 'unselected', 'cases', 'case', 'families', 'breast', 'cancer', 'model', 'involved', 'rare', 'highly', 'penetrant', 'recognized', 'since', 'cloned', 'patients', 'screened', 

2.4 Entity Recognition

In [15]:
import spacy

# Load spaCy's small English model
nlp = spacy.load('en_core_web_sm')

# Function for Named Entity Recognition (NER)
def perform_ner(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

# Perform NER on each preprocessed text
# Here, we're joining the preprocessed tokens back into a single string
entities_per_doc = {filename: perform_ner(" ".join(tokens)) for filename, tokens in preprocessed_texts.items()}

# Print the named entities from the first file as an example
first_file = list(entities_per_doc.keys())[0]  # Get the name of the first file
print(f"Named entities for {first_file}:")
print(entities_per_doc[first_file])

Named entities for Breast-cancer-susceptibility&#x2014;A-new-look-at-.pdf:
[('p r e v e w', 'ORG'), ('quarter', 'DATE'), ('six years', 'DATE'), ('two', 'CARDINAL'), ('half', 'CARDINAL'), ('dong spans', 'PERSON'), ('hemminki', 'GPE'), ('hundred', 'CARDINAL'), ('dozen', 'CARDINAL'), ('june', 'DATE'), ('next year', 'DATE'), ('two', 'CARDINAL'), ('sutton', 'ORG'), ('london', 'GPE'), ('precise london', 'ORG'), ('united kingdom', 'GPE'), ('dong hemminki', 'PERSON'), ('zimmern easton', 'PERSON'), ('june', 'DATE')]


2.5 Stemming/Lemmatization

In [16]:
# Using PorterStemmer for Stemming
from nltk.stem import PorterStemmer
import spacy

# Initializing the PorterStemmer
stemmer = PorterStemmer()

# Function to stem tokens
def stem_tokens(tokens):
    return [stemmer.stem(token) for token in tokens]

# Applying stemming to each document in preprocessed_texts
# Storing the stemmed tokens in a dictionary with filenames as keys
stemmed_texts = {filename: stem_tokens(tokens) for filename, tokens in preprocessed_texts.items()}

# Example output for stemming
first_file = list(preprocessed_texts.keys())[0]  # Get the first file name
print(f"Stemmed tokens for {first_file}:")
print(stemmed_texts[first_file])

Stemmed tokens for Breast-cancer-susceptibility&#x2014;A-new-look-at-.pdf:
['p', 'r', 'e', 'v', 'e', 'w', 'breast', 'cancer', 'new', 'look', 'old', 'model', 'polygen', 'model', 'mani', 'individu', 'weak', 'gene', 'combin', 'multipl', 'caus', 'rang', 'risk', 'popul', 'explain', 'sever', 'puzzl', 'aspect', 'famili', 'breast', 'cancer', 'epidemiolog', 'includ', 'high', 'risk', 'famili', 'failur', 'identifi', 'import', 'new', 'gene', 'sinc', 'discoveri', 'germlin', 'mutat', 'model', 'develop', 'segreg', 'sever', 'quit', 'differ', 'genet', 'model', 'account', 'larg', 'analysi', 'seri', 'provid', 'statist', 'satisfactori', 'fit', 'breast', 'cancer', 'famili', 'famili', 'collect', 'multipl', 'famili', 'data', 'origin', 'cash', 'cent', 'unselect', 'case', 'case', 'famili', 'breast', 'cancer', 'model', 'involv', 'rare', 'highli', 'penetr', 'recogn', 'sinc', 'clone', 'patient', 'screen', 'domin', 'gene', 'clau', 'et', 'known', 'gene', 'antoni', 'et', 'model', 'combin', 'brcagen', 'account', 'les

In [17]:
# Importing spaCy for Lemmatization
# Loading the spaCy model
nlp = spacy.load('en_core_web_sm')

# Function to lemmatize tokens
def lemmatize_tokens(tokens):
    return [token.lemma_ for token in nlp(" ".join(tokens)) if token.is_alpha]

# Applying lemmatization to each document in preprocessed_texts
# Storing the lemmatized tokens in a dictionary with filenames as keys
lemmatized_texts = {filename: lemmatize_tokens(tokens) for filename, tokens in preprocessed_texts.items()}

# Example output for lemmatization
print(f"Lemmatized tokens for {first_file}:")
print(lemmatized_texts[first_file])

Lemmatized tokens for Breast-cancer-susceptibility&#x2014;A-new-look-at-.pdf:
['p', 'r', 'e', 'v', 'e', 'w', 'breast', 'cancer', 'new', 'look', 'old', 'model', 'polygenic', 'model', 'many', 'individually', 'weak', 'gene', 'combine', 'multiplicately', 'cause', 'range', 'risk', 'population', 'explain', 'several', 'puzzle', 'aspect', 'familial', 'breast', 'cancer', 'epidemiology', 'include', 'high', 'risk', 'family', 'failure', 'identify', 'important', 'new', 'gene', 'since', 'discovery', 'germline', 'mutation', 'model', 'develop', 'segregation', 'several', 'quite', 'different', 'genetic', 'model', 'account', 'large', 'analysis', 'series', 'provide', 'statistically', 'satisfactory', 'fit', 'breast', 'cancer', 'family', 'family', 'collection', 'multiple', 'family', 'datum', 'original', 'cash', 'cent', 'unselecte', 'case', 'case', 'family', 'breast', 'cancer', 'model', 'involve', 'rare', 'highly', 'penetrant', 'recognize', 'since', 'clone', 'patient', 'screen', 'dominant', 'gene', 'claus', 

2.6 Metadata Extraction

In [27]:
import PyPDF2

# Function to extract metadata from a PDF
def extract_metadata(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)  # Use PdfReader instead of PdfFileReader
        metadata = reader.metadata  # Access metadata directly
        return metadata

# Dictionary to store metadata for each PDF
pdf_metadata = {}

# Extract metadata from all uploaded PDFs
for filename in uploaded.keys():
    pdf_metadata[filename] = extract_metadata(filename)

# Example: Print the metadata of the first file
if pdf_metadata:  # Check if any metadata was extracted
    first_file = list(pdf_metadata.keys())[0]  # Get the first file name
    print(f"Metadata for {first_file}:")
    print(pdf_metadata[first_file])
else:
    print("No metadata found.")


Metadata for Breast-cancer-susceptibility&#x2014;A-new-look-at- (1).pdf:
{'/CreationDate': 'D:20020605163758', '/Producer': 'Acrobat Distiller 4.05 for Windows', '/Author': 'KThompso', '/Creator': 'ADOBEPS4.DRV Version 4.41', '/Title': 'PII: S1535-6108(02)00079-X', '/ModDate': "D:20040310175832+05'30'"}


In [25]:
print(uploaded.keys())  # This should print the filenames of the uploaded files

dict_keys(['Breast-cancer-susceptibility&#x2014;A-new-look-at- (1).pdf', 'Integrated-breast-cancer-genomics_ccell (1).pdf'])
